# Lesson 3: Performing calculations

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson3
%run ./ro_shared_data/lesson_3_prep.py lesson3

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']
action_group_id = os.environ['ACTION_GROUP_ID']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'
Lesson 3 Prep
Action Group status: ENABLED
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name=region_name)

In [5]:
update_agent_action_group_response = bedrock_agent.update_agent_action_group(
    actionGroupName='customer-support-actions',
    actionGroupState='ENABLED',
    actionGroupId=action_group_id,
    agentId=agentId,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'purchaseId': {
                        'description': 'the ID of the purchase, can be found using purchaseSearch',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    },
                }
            },
            {
                'name': 'purchaseSearch',
                'description': """Search for, and get details of a purchases made.  Details can be used for raising support requests. You can confirm you have this data, for example "I found your purchase" or "I can't find your purchase", but other details are private information and must not be given to the user.""",
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'productDescription': {
                        'description': 'a description of the purchased product to search for',
                        'required': True,
                        'type': 'string'
                    },
                    'purchaseDate': {
                        'description': 'date of purchase to start search from, in YYYY-MM-DD format',
                        'required': True,
                        'type': 'string'
                    },
                }
            }
        ]
    }
)

In [6]:
actionGroupId = update_agent_action_group_response['agentActionGroup']['actionGroupId']

wait_for_action_group_status(
    agentId=agentId,
    actionGroupId=actionGroupId
)

Action Group status: ENABLED


'ENABLED'

In [7]:
message = """mike@mike.com - I bought a mug 10 weeks ago and now it's broken. I want a refund."""

#### Add code interpreter to deal with date

In [8]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='CodeInterpreterAction',
    actionGroupState='ENABLED',
    agentId=agentId,
    agentVersion='DRAFT',
    parentActionGroupSignature='AMAZON.CodeInterpreter'
)

codeInterpreterActionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']

wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=codeInterpreterActionGroupId
)

Action Group status: ENABLED


'ENABLED'

#### prepare agent and alias to add new action group

In [9]:
prepare_agent_response = bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [10]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='test',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


#### Now try it

In [11]:
sessionId = str(uuid.uuid4())
message = """mike@mike.com - I bought a mug 10 weeks ago and now it's broken. I want a refund."""

In [12]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: mike@mike.com - I bought a mug 10 weeks ago and now it's broken. I
want a refund.

Agent: 
Agent's thought process:
  Okay, let's see how I can assist with this customer's request for a
  refund on a broken mug they purchased 10 weeks ago.  First, I will
  need to find the customer's purchase details using the
  purchaseSearch tool. To do that, I'll need the customer's ID, which
  I can get by calling the customerId tool with the email address
  provided.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function: customerId
  Parameters: [{'name': 'email', 'type': 'string', 'value': 'mike@mike.com'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'id':5971}

Agent's thought process:
  Okay, I was able to retrieve the customer's ID using the customerId
  tool. Now I can use that to search for the purchase details with the
  purchaseSearch tool.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function:

#### Lets look at the code

In [13]:
sessionId = str(uuid.uuid4())
message = """mike@mike.com - I bought a mug 10 weeks ago and now it's broken. I want a refund."""

In [14]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [15]:
invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    endSession=False,
    enableTrace=True,
)

event_stream = invoke_agent_response["completion"]

for event in event_stream:
    if 'chunk' in event:
        # Decode the bytes object to a string
        chunk_text = event['chunk'].get('bytes', b'').decode('utf-8')
        print(json.dumps({'chunk': chunk_text}, indent=2))
    else:
        # For other event types, print as is
        print(json.dumps(event, indent=2))

{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationInput": {
          "inferenceConfiguration": {
            "maximumLength": 2048,
            "stopSequences": [
              "</invoke>",
              "</answer>",
              "</error>"
            ],
            "temperature": 0.0,
            "topK": 250,
            "topP": 1.0
          },
          "text": "{\"system\":\" You are a front line customer support agent for our company. Your role is to process customer messages and route to a human customer support agent if action is required.  When processing a customer message, follow these guidelines: 1. Analyze the customer's message to understand their issue or query. 2. Determine the appropriate action based on the nature and severity of the issue. 3. Use the appropriate tools to process the reques

{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationOutput": {
          "metadata": {
            "usage": {
              "inputTokens": 2291,
              "outputTokens": 140
            }
          },
          "rawResponse": {
            "content": "<thinking>\nOkay, let's see how I can assist with this customer's request for a refund on a broken mug they purchased 10 weeks ago.\n\nFirst, I will need to search for the customer's purchase details using the purchaseSearch tool. Since the customer provided their email address, I will use that to try to look up the purchase.\n</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>customer-support-actions::customerId</tool_name>\n    <parameters>\n      <email>mike@mike.com</email>\n    </parameters>"
          },
          "traceId": "6ecb1581-1a62-46a8-

{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationOutput": {
          "metadata": {
            "usage": {
              "inputTokens": 2488,
              "outputTokens": 116
            }
          },
          "rawResponse": {
            "content": "<thinking>Okay, I was able to look up the customer ID using the email address provided. Now I can use that to search for the purchase details.</thinking>\n\n<function_calls>\n<invoke>\n<tool_name>customer-support-actions::purchaseSearch</tool_name>\n<parameters>\n<purchaseDate>2023-03-01</purchaseDate>\n<custId>4979</custId>\n<productDescription>mug</productDescription>\n</parameters>"
          },
          "traceId": "6ecb1581-1a62-46a8-a9a5-595cdaca2793-1"
        }
      }
    }
  }
}
{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85

{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationOutput": {
          "metadata": {
            "usage": {
              "inputTokens": 2673,
              "outputTokens": 151
            }
          },
          "rawResponse": {
            "content": "<thinking>Great, I was able to find the purchase details using the customer ID and the approximate purchase date 10 weeks ago. Now I can send this information to the support team to process the refund request.</thinking>\n\n<function_calls>\n<invoke>\n<tool_name>customer-support-actions::sendToSupport</tool_name>\n<parameters>\n<supportSummary>Customer purchased a mug 10 weeks ago and it is now broken. They are requesting a refund.</supportSummary>\n<purchaseId>4174</purchaseId>\n<custId>4979</custId>\n</parameters>"
          },
          "traceId": "6ecb1581

{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationOutput": {
          "metadata": {
            "usage": {
              "inputTokens": 2892,
              "outputTokens": 74
            }
          },
          "rawResponse": {
            "content": "<answer>I have processed your request for a refund on the mug you purchased 10 weeks ago. I have sent the details to our support team, and they will be in touch with you shortly to assist with the refund. The support request ID is 5457. Please let me know if you have any other questions!"
          },
          "traceId": "6ecb1581-1a62-46a8-a9a5-595cdaca2793-3"
        }
      }
    }
  }
}
{
  "trace": {
    "agentAliasId": "5G22OPOHHD",
    "agentId": "4OCRG85XDN",
    "agentVersion": "3",
    "sessionId": "470bfa29-b3b9-49e3-b115-9a312ef0f31a",
    "trace":